# Code structure

This notebook describes the general code structure of `pyHYBRO`.

## 1.) The Problem

The problem contains all objective functions (+ potentially grad/hess) + optimization bounds

In [8]:
import pyhybro
import numpy as np

fun = lambda x : np.sum(x**2)
grad = lambda x: 2*x
hess = lambda x: 2* np.identity(x.size)

lb = np.array([-2, -2])
ub = np.array([2, 2])

problem = pyhybro.Problem(fun=fun, 
                          grad=grad, 
                          hess=hess, 
                          lb=lb, 
                          ub=ub)

## 2.) The OptimizationManager 

The `OptimizationManager` basically does all the scheduling between the different optimizers (I mean, we do not only need to cycle between local&global optimizers. It might be also interesting to look which  optimizers perform well in the beginning/the end of a local optimization run and change between them etc. ... So I wanted to keep that flexible :) ).

The `OptimizationManagerBase` defines three functions:

* `minimize(...)` Takes a `problem` + `reference_set` and minimizes. Returns a `OptimizationResult`
* `get_next_optimizer` returns the next optimizer, that shall be used
* `check_termination` ... Checks termination of optimization.


## 3.) OptimizationResult

The `OptimizationResult` shall save the current `ReferenceSet` and meta-information as number of function/gradient/hessian evaluation. Potentially also more. The `OptimizationResult` is created inside of `OptimizationManager.minimize` function and handed to the individual Optimizers, which update the `OptimizationResult`

## 4.) ReferenceSet

The `ReferenceSet` is a list of `ReferenceSetMembers`. I implemented it in a way, that it allows for additional functionality, but individual members can be accessed via `ReferenceSet[idx]`. 

## 5.) Optimizer

Inherit from `OptimizerBase`. Options are given as a dict.  `Optimizer.minimize` takes a `Problem` and an `OptimizationResult` and returns an updated `OptimizationResult`.

# How to implement ESS:

We would need to implement the `OptimizationManager` scheme, which to me looks like only switching between a local and a global optimizer + implementing a stopping criterion. Further we need to implement the global Optimizer + interface a local one. Potentially via Scipy/Fides/pyPESTO.

## Syntax goal:

```python
import pyhybro

reference_set = ... # We need methods to easily sample initial values from a problem instance...

local_optimizer = pyhybro.ScipyOptimizer(options={maxiter=200})
global_optimizer = pyhybro.EssGlobal()

optimization_manager = pyhybro.EssManager(local_optimizer = local_optimizer, 
                                          global_optimizer = global_optimizer, 
                                          options = {max_switches = 50})

result = optimization_manager.minimize(problem, 
                                       reference_set)

```